In [1]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

import os
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.utils.data as Data
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
seed = 2022
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
hla_sequence = pd.read_csv('../data/other/HLAI_pseudosequences_34mer.csv')

In [79]:
def make_data(data):
    pep_inputs, hla_inputs, labels = [], [], []
    pep_lens = []
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
#         pep_lens.append(len(pep)+34)
        pep_lens.append(49)
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
        labels.append(label)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs), torch.LongTensor(labels), torch.LongTensor(pep_lens)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels, pep_lens):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels
        self.pep_lens = pep_lens

    def __len__(self): # 样本数
        return self.pep_inputs.shape[0] # 改成hla_inputs也可以哦！

    def __getitem__(self, idx):
#         return self.pep_inputs[idx], self.hla_inputs[idx], self.labels[idx],self.pep_lens[idx]
        return torch.cat((self.hla_inputs[idx],self.pep_inputs[idx]), dim=0), self.labels[idx], self.pep_lens[idx]

def seq_len_to_mask(seq_len, max_len=49): #50
    r"""
    将一个表示sequence length的一维数组转换为二维的mask，不包含的位置为0。
    转变 1-d seq_len到2-d mask.
    .. code-block::
    
        >>> seq_len = torch.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len)
        >>> print(mask.size())
        torch.Size([14, 15])
        >>> seq_len = np.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len)
        >>> print(mask.shape)
        (14, 15)
        >>> seq_len = torch.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len, max_len=100)
        >>>print(mask.size())
        torch.Size([14, 100])
    :param np.ndarray,torch.LongTensor seq_len: shape将是(B,)
    :param int max_len: 将长度pad到这个长度。默认(None)使用的是seq_len中最长的长度。但在nn.DataParallel的场景下可能不同卡的seq_len会有
        区别，所以需要传入一个max_len使得mask的长度是pad到该长度。
    :return: np.ndarray, torch.Tensor 。shape将是(B, max_length)， 元素类似为bool或torch.uint8
    """
    if isinstance(seq_len, np.ndarray):
        assert len(np.shape(seq_len)) == 1, f"seq_len can only have one dimension, got {len(np.shape(seq_len))}."
        max_len = int(max_len) if max_len else int(seq_len.max())
        broad_cast_seq_len = np.tile(np.arange(max_len), (len(seq_len), 1))
        mask = broad_cast_seq_len < seq_len.reshape(-1, 1)

    elif isinstance(seq_len, torch.Tensor):
        assert seq_len.dim() == 1, f"seq_len can only have one dimension, got {seq_len.dim() == 1}."
        batch_size = seq_len.size(0)
        max_len = int(max_len) if max_len else seq_len.max().long()
        broad_cast_seq_len = torch.arange(max_len).expand(batch_size, -1).to(seq_len)
        mask = broad_cast_seq_len.lt(seq_len.unsqueeze(1))
    else:
        raise TypeError("Only support 1-d numpy.ndarray or 1-d torch.Tensor.")

    return mask

def get_embeddings(init_embed, padding_idx=None):
    r"""
    根据输入的init_embed返回Embedding对象。如果输入是tuple, 则随机初始化一个nn.Embedding; 如果输入是numpy.ndarray, 则按照ndarray
    的值将nn.Embedding初始化; 如果输入是torch.Tensor, 则按该值初始化nn.Embedding; 如果输入是fastNLP中的embedding将不做处理
    返回原对象。
    :param init_embed: 可以是 tuple:(num_embedings, embedding_dim), 即embedding的大小和每个词的维度;也可以传入
        nn.Embedding 对象, 此时就以传入的对象作为embedding; 传入np.ndarray也行，将使用传入的ndarray作为作为Embedding初始化;
        传入torch.Tensor, 将使用传入的值作为Embedding初始化。
    :param padding_idx: 当传入tuple时，padding_idx有效
    :return nn.Embedding:  embeddings
    """
    if isinstance(init_embed, tuple):
        res = nn.Embedding(num_embeddings=init_embed[0], embedding_dim=init_embed[1], padding_idx=padding_idx)
#         nn.init.uniform_(res.weight.data, a=-np.sqrt(3 / res.weight.data.size(1)),
#                          b=np.sqrt(3 / res.weight.data.size(1)))
    elif isinstance(init_embed, nn.Module):
        res = init_embed
    elif isinstance(init_embed, torch.Tensor):
        res = nn.Embedding.from_pretrained(init_embed, freeze=False)
    elif isinstance(init_embed, np.ndarray):
        init_embed = torch.tensor(init_embed, dtype=torch.float32)
        res = nn.Embedding.from_pretrained(init_embed, freeze=False)
    else:
        raise TypeError(
            'invalid init_embed type: {}'.format((type(init_embed))))
    return res

In [139]:

class StarTransformer(nn.Module):
    r"""
    Star-Transformer 的encoder部分。 输入3d的文本输入, 返回相同长度的文本编码
    paper: https://arxiv.org/abs/1902.09113
    """

    def __init__(self, hidden_size, num_layers, num_head, head_dim, dropout=0.1, max_len=None):
        r"""
        
        :param int hidden_size: 输入维度的大小。同时也是输出维度的大小。
        :param int num_layers: star-transformer的层数
        :param int num_head: head的数量。
        :param int head_dim: 每个head的维度大小。
        :param float dropout: dropout 概率. Default: 0.1
        :param int max_len: int or None, 如果为int，输入序列的最大长度，
            模型会为输入序列加上position embedding。
            若为`None`，忽略加上position embedding的步骤. Default: `None`
        """
        super(StarTransformer, self).__init__()
        self.iters = num_layers

        self.norm = nn.ModuleList([nn.LayerNorm(hidden_size, eps=1e-6) for _ in range(self.iters)])
        # self.emb_fc = nn.Conv2d(hidden_size, hidden_size, 1)
        self.emb_drop = nn.Dropout(dropout)
        self.ring_att = nn.ModuleList(
            [_MSA1(hidden_size, nhead=num_head, head_dim=head_dim, dropout=0.0)
             for _ in range(self.iters)])
        self.star_att = nn.ModuleList(
            [_MSA2(hidden_size, nhead=num_head, head_dim=head_dim, dropout=0.0)
             for _ in range(self.iters)])

        if max_len is not None:
            self.pos_emb = nn.Embedding(max_len, hidden_size)
        else:
            self.pos_emb = None

    def forward(self, data, mask):
        r"""
        :param FloatTensor data: [batch, length, hidden] 输入的序列
        :param ByteTensor mask: [batch, length] 输入序列的padding mask, 在没有内容(padding 部分) 为 0,
            否则为 1
        :return: [batch, length, hidden] 编码后的输出序列
                [batch, hidden] 全局 relay 节点, 详见论文
        """

        def norm_func(f, x):
            # B, H, L, 1
            return f(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        B, L, H = data.size()
        mask = (mask.eq(False))  # flip the mask for masked_fill_
        smask = torch.cat([torch.zeros(B, 1, ).byte().to(mask), mask], 1)

        embs = data.permute(0, 2, 1)[:, :, :, None]  # B H L 1
        if self.pos_emb:
            P = self.pos_emb(torch.arange(L, dtype=torch.long, device=embs.device) \
                             .view(1, L)).permute(0, 2, 1).contiguous()[:, :, :, None]  # 1 H L 1
            embs = embs + P
        embs = norm_func(self.emb_drop, embs)
        nodes = embs
        relay = embs.mean(2, keepdim=True)
        ex_mask = mask[:, None, :, None].expand(B, H, L, 1)
        r_embs = embs.view(B, H, 1, L)
#         nodes_attns = []
#         relays_attns = []
        for i in range(self.iters):
            ax = torch.cat([r_embs, relay.expand(B, H, 1, L)], 2)
            nodes = F.leaky_relu(self.ring_att[i](norm_func(self.norm[i], nodes), ax=ax))
            # nodes = F.leaky_relu(self.ring_att[i](nodes, ax=ax))
#             nodes_attns.append(nodes_att)
            relay = F.leaky_relu(self.star_att[i](relay, torch.cat([relay, nodes], 2), smask))
#             relays_attns.append(relay_att)
            nodes = nodes.masked_fill_(ex_mask, 0)

        nodes = nodes.view(B, H, L).permute(0, 2, 1)

        return nodes, relay.view(B, H)#, nodes_attns, relays_attns


class _MSA1(nn.Module):
    def __init__(self, nhid, nhead=10, head_dim=10, dropout=0.1):
        super(_MSA1, self).__init__()
        # Multi-head Self Attention Case 1, doing self-attention for small regions
        # Due to the architecture of GPU, using hadamard production and summation are faster than dot production when unfold_size is very small
        self.WQ = nn.Conv2d(nhid, nhead * head_dim, 1)
        self.WK = nn.Conv2d(nhid, nhead * head_dim, 1)
        self.WV = nn.Conv2d(nhid, nhead * head_dim, 1)
        self.WO = nn.Conv2d(nhead * head_dim, nhid, 1)

        self.drop = nn.Dropout(dropout)

        self.nhid, self.nhead, self.head_dim, self.unfold_size = nhid, nhead, head_dim, 3

    def forward(self, x, ax=None):
        # x: B, H, L, 1, ax : B, H, X, L append features
        nhid, nhead, head_dim, unfold_size = self.nhid, self.nhead, self.head_dim, self.unfold_size
        B, H, L, _ = x.shape

        q, k, v = self.WQ(x), self.WK(x), self.WV(x)  # x: (B,H,L,1)

        if ax is not None:
            aL = ax.shape[2]
            ak = self.WK(ax).view(B, nhead, head_dim, aL, L)
            av = self.WV(ax).view(B, nhead, head_dim, aL, L)
        q = q.view(B, nhead, head_dim, 1, L)
        k = F.unfold(k.view(B, nhead * head_dim, L, 1), (unfold_size, 1), padding=(unfold_size // 2, 0)) \
            .view(B, nhead, head_dim, unfold_size, L)
        v = F.unfold(v.view(B, nhead * head_dim, L, 1), (unfold_size, 1), padding=(unfold_size // 2, 0)) \
            .view(B, nhead, head_dim, unfold_size, L)
        if ax is not None:
            k = torch.cat([k, ak], 3)
            v = torch.cat([v, av], 3)

        alphas = self.drop(F.softmax((q * k).sum(2, keepdim=True) / np.sqrt(head_dim), 3))  # B N L 1 U
        att = (alphas * v).sum(3).view(B, nhead * head_dim, L, 1)

        ret = self.WO(att)

        return ret #,alphas


class _MSA2(nn.Module):
    def __init__(self, nhid, nhead=10, head_dim=10, dropout=0.1):
        # Multi-head Self Attention Case 2, a broadcastable query for a sequence key and value
        super(_MSA2, self).__init__()
        self.WQ = nn.Conv2d(nhid, nhead * head_dim, 1)
        self.WK = nn.Conv2d(nhid, nhead * head_dim, 1)
        self.WV = nn.Conv2d(nhid, nhead * head_dim, 1)
        self.WO = nn.Conv2d(nhead * head_dim, nhid, 1)

        self.drop = nn.Dropout(dropout)

        self.nhid, self.nhead, self.head_dim, self.unfold_size = nhid, nhead, head_dim, 3
    def forward(self, x, y, mask=None):
        # x: B, H, 1, 1, 1 y: B H L 1
        nhid, nhead, head_dim, unfold_size = self.nhid, self.nhead, self.head_dim, self.unfold_size
        B, H, L, _ = y.shape

        q, k, v = self.WQ(x), self.WK(y), self.WV(y)

        q = q.view(B, nhead, 1, head_dim)  # B, H, 1, 1 -> B, N, 1, h
        k = k.view(B, nhead, head_dim, L)  # B, H, L, 1 -> B, N, h, L
        v = v.view(B, nhead, head_dim, L).permute(0, 1, 3, 2)  # B, H, L, 1 -> B, N, L, h
        pre_a = torch.matmul(q, k) / np.sqrt(head_dim)
        if mask is not None:
            pre_a = pre_a.masked_fill(mask[:, None, None, :], -float('inf'))
        alphas = self.drop(F.softmax(pre_a, 3))  # B, N, 1, L
        att = torch.matmul(alphas, v).view(B, -1, 1, 1)  # B, N, 1, h -> B, N*h, 1, 1
        return self.WO(att) #,alphas
    
class StarTransEnc(nn.Module):
    r"""
    带word embedding的Star-Transformer Encoder
    """

    def __init__(self, embed,
                 hidden_size,
                 num_layers,
                 num_head,
                 head_dim,
                 max_len,
                 emb_dropout,
                 dropout):
        r"""
        
        :param embed: 单词词典, 可以是 tuple, 包括(num_embedings, embedding_dim), 即
            embedding的大小和每个词的维度. 也可以传入 nn.Embedding 对象,此时就以传入的对象作为embedding
        :param hidden_size: 模型中特征维度.
        :param num_layers: 模型层数.
        :param num_head: 模型中multi-head的head个数.
        :param head_dim: 模型中multi-head中每个head特征维度.
        :param max_len: 模型能接受的最大输入长度.
        :param emb_dropout: 词嵌入的dropout概率.
        :param dropout: 模型除词嵌入外的dropout概率.
        """
        super(StarTransEnc, self).__init__()
        self.embedding = get_embeddings(embed,padding_idx=0)
        emb_dim = self.embedding.embedding_dim
        self.emb_fc = nn.Linear(emb_dim, hidden_size)
        # self.emb_drop = nn.Dropout(emb_dropout)
        self.encoder = StarTransformer(hidden_size=hidden_size,
                                       num_layers=num_layers,
                                       num_head=num_head,
                                       head_dim=head_dim,
                                       dropout=dropout,
                                       max_len=max_len)
        
#         conv_block_klass = ConvBlock
# #         Embedding Layer
#         self.stem = nn.Sequential(
#         #             Rearrange('b n d -> b d n'),
# #             Dynamic_conv1d(49, 49, 3,padding = 1),
#             Residual(conv_block_klass(49)),
# #             AttentionPool(49, pool_size = 2)
            
#         )
#         self.stem2 = nn.Sequential(
#         #             Rearrange('b n d -> b d n'),
#             nn.Conv1d(34, 34, 3,padding = 1),
#             Residual(conv_block_klass(34)),
#             AttentionPool(34, pool_size = 2)
#         )

    def forward(self, x, mask):
        r"""
        :param FloatTensor x: [batch, length, hidden] 输入的序列
        :param ByteTensor mask: [batch, length] 输入序列的padding mask, 在没有内容(padding 部分) 为 0,
            否则为 1
        :return: [batch, length, hidden] 编码后的输出序列
                [batch, hidden] 全局 relay 节点, 详见论文
        """
        x = self.embedding(x)
        x = self.emb_fc(x)
#         x = self.stem(x)
        #nodes, relay, nodes_attns, relays_attns = self.encoder(x3, mask3)
        nodes, relay = self.encoder(x, mask)
        return nodes, relay, #nodes_attns, relays_attns


class _Cls(nn.Module):
    def __init__(self, in_dim, num_cls, hid_dim, dropout=0.1):
        super(_Cls, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(hid_dim, num_cls),
        )

    def forward(self, x):
        h = self.fc(x)
        return h

class STSeqCls(nn.Module):
    r"""
    用于分类任务的Star-Transformer
    """

    def __init__(self, embed, num_cls=2,
                 hidden_size=300,
                 num_layers=1,
                 num_head=9,
                 head_dim=32,
                 max_len=512,
                 cls_hidden_size=600,
                 emb_dropout=0.1,
                 dropout=0.1):
        r"""
        
        :param embed: 单词词典, 可以是 tuple, 包括(num_embedings, embedding_dim), 即
            embedding的大小和每个词的维度. 也可以传入 nn.Embedding 对象, 此时就以传入的对象作为embedding
        :param num_cls: 输出类别个数
        :param hidden_size: 模型中特征维度. Default: 300
        :param num_layers: 模型层数. Default: 4
        :param num_head: 模型中multi-head的head个数. Default: 8
        :param head_dim: 模型中multi-head中每个head特征维度. Default: 32
        :param max_len: 模型能接受的最大输入长度. Default: 512
        :param cls_hidden_size: 分类器隐层维度. Default: 600
        :param emb_dropout: 词嵌入的dropout概率. Default: 0.1
        :param dropout: 模型除词嵌入外的dropout概率. Default: 0.1
        """
        super(STSeqCls, self).__init__()
        self.enc = StarTransEnc(embed=embed,
                                hidden_size=hidden_size,
                                num_layers=num_layers,
                                num_head=num_head,
                                head_dim=head_dim,
                                max_len=max_len,
                                emb_dropout=emb_dropout,
                                dropout=dropout)
        self.cls = _Cls(hidden_size, num_cls, cls_hidden_size, dropout=dropout)


    def forward(self, words, seq_len):
        r"""
        :param words: [batch, seq_len] 输入序列
        :param seq_len: [batch,] 输入序列的长度
        :return output: [batch, num_cls] 输出序列的分类的概率
        """
        mask = seq_len_to_mask(seq_len,max_len=49).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
#         mask2 = seq_len_to_mask(torch.tensor([34]*len(seq_len))).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        nodes, relay = self.enc(words, mask)
        y = 0.5 * (relay + nodes.max(1)[0])
#         y = torch.cat([relay, torch.sort(nodes,dim=1)[0][:,-1,:], torch.sort(nodes,dim=1)[0][:,-2,:]],1)
        
        output = self.cls(y)  # [bsz, n_cls]
        return output#, nodes_attns, relays_attns

In [106]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)


# In[25]:


def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


# In[26]:


f_mean = lambda l: sum(l)/len(l)


# In[28]:


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd

In [107]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list = []
    for train_pep_inputs, train_pep_lens, train_labels in tqdm(train_loader):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        train_pep_inputs, train_labels = train_pep_inputs.to(device), train_labels.to(device)
        train_pep_lens = train_pep_lens.to(device)
        t1 = time.time()
        train_outputs = model(train_pep_inputs, train_pep_lens)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss.item())
#         relays_attns_list.append(relays_attns)
#         nodes_attns_list.append(nodes_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, relays_attns_list, nodes_attns_list


# In[30]:


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    model.eval()
    torch.manual_seed(2022)
    torch.cuda.manual_seed(2022)
    with torch.no_grad():
        loss_val_list = []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_pep_lens, val_labels in tqdm(val_loader):
            val_pep_inputs, val_labels = val_pep_inputs.to(device), val_labels.to(device)
            val_pep_lens = val_pep_lens.to(device)
            val_outputs = model(val_pep_inputs, val_pep_lens)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss.item())

            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list, print_ = True)
    return ys_val, loss_val_list, metrics_val


In [108]:
pep_max_len = 15 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

# vocab = np.load('./vocab_dict.npy', allow_pickle = True).item()
vocab = {'-': 0,
 'Y': 1,
 'A': 2,
 'T': 3,
 'V': 4,
 'L': 5,
 'D': 6,
 'E': 7,
 'G': 8,
 'R': 9,
 'H': 10,
 'I': 11,
 'W': 12,
 'Q': 13,
 'K': 14,
 'M': 15,
 'F': 16,
 'N': 17,
 'S': 18,
 'P': 19,
 'C': 20}
vocab_size = len(vocab)


n_layers = 1  # number of Encoder of Decoder Layer
n_heads = 8

batch_size = 1024
epochs = 25
threshold = 0.5

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [109]:
#G:TransPHLA-AOMP/Dataset/
def data_with_loader(type_ = 'train',fold = None,  batch_size = 128):
    if type_ != 'train' and type_ != 'val':
        data = pd.read_csv('../data/test_set/{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.read_csv('../data/train_set/NetMHCpan4.1/train_data_fold{}.csv'.format(fold)) #, index_col = 0
    elif type_ == 'val':
        data = pd.read_csv('../data/train_set/NetMHCpan4.1/val_data_fold{}.csv'.format(fold)) #, index_col = 0
        
    pep_inputs, hla_inputs, labels, pep_lens = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, pep_lens, labels), batch_size, shuffle = False, num_workers = 0)
    
    return data, pep_inputs, hla_inputs, pep_lens, labels, loader

In [110]:
independent_data, independent_pep_inputs, independent_hla_inputs, independent_pep_lens, independent_labels, independent_loader = data_with_loader(type_ = 'independent',fold = None,  batch_size = batch_size)
external_data, external_pep_inputs, external_hla_inputs, external_pep_lens, external_labels, external_loader = data_with_loader(type_ = 'external',fold = None,  batch_size = batch_size)

In [111]:
ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list = [], []
independent_fold_metrics_list, external_fold_metrics_list, ys_independent_fold_dict, ys_external_fold_dict = [], [], {}, {}
attns_train_fold_dict, attns_val_fold_dict, attns_independent_fold_dict, attns_external_fold_dict = {}, {}, {}, {}
loss_train_fold_dict, loss_val_fold_dict, loss_independent_fold_dict, loss_external_fold_dict = {}, {}, {}, {}

for fold in range(3,4):
    print('=====Fold-{}====='.format(fold))
    print('-----Generate data loader-----')
    train_data, train_pep_inputs, train_hla_inputs, train_pep_lens, train_labels, train_loader = data_with_loader(type_ = 'train', fold = fold,  batch_size = batch_size)
    val_data, val_pep_inputs, val_hla_inputs, val_pep_lens, val_labels, val_loader = data_with_loader(type_ = 'val', fold = fold,  batch_size = batch_size)
    print('Fold-{} Label info: Train = {} | Val = {}'.format(fold, Counter(train_data.label), Counter(val_data.label)))

    print('-----Compile model-----')
    model = STSeqCls((21, 100), num_cls=2, hidden_size=300, num_layers=1, num_head=8, max_len=49,cls_hidden_size=600,dropout=0.1,head_dim=32).to(device)
    print(model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)#, momentum = 0.99)
#     optimizer = ScheduledOptim(optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09), 2, 64, 10)

    print('-----Train-----')
    dir_saver = 'G:TransPHLA-AOMP/model/STformer/'
    path_saver = 'G:TransPHLA-AOMP/model/STformer/netmhcpan/st_layer{}_multihead{}_fold{}_netmhcpan.pkl'.format(n_layers, n_heads, fold)
    print('dir_saver: ', dir_saver)
    print('path_saver: ', path_saver)

    metric_best, ep_best = 0, -1
    time_train = 0
    for epoch in range(1, epochs + 1):

        ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
        ys_val, loss_val_list, metrics_val = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

        metrics_ep_avg = sum(metrics_val[:4])/4
        if metrics_ep_avg > metric_best: 
            metric_best, ep_best = metrics_ep_avg, epoch
            if not os.path.exists(dir_saver):
                os.makedirs(dir_saver)
            print('****Saving model: Best epoch = {} | 5metrics_Best_avg = {:.4f}'.format(ep_best, metric_best))
            print('*****Path saver: ', path_saver)
            torch.save(model.eval().state_dict(), path_saver)

        time_train += time_train_ep

    print('-----Optimization Finished!-----')
    print('-----Evaluate Results-----')
    if ep_best >= 0:
        print('*****Path saver: ', path_saver)
        model.load_state_dict(torch.load(path_saver))
        model_eval = model.eval()

        ys_res_train, loss_res_train_list, metrics_res_train = eval_step(model_eval, train_loader, fold, ep_best, epochs, use_cuda) # , train_res_attns
        ys_res_val, loss_res_val_list, metrics_res_val = eval_step(model_eval, val_loader, fold, ep_best, epochs, use_cuda) # , val_res_attns
        ys_res_independent, loss_res_independent_list, metrics_res_independent = eval_step(model_eval, independent_loader, fold, ep_best, epochs, use_cuda) # , independent_res_attns
        ys_res_external, loss_res_external_list, metrics_res_external = eval_step(model_eval, external_loader, fold, ep_best, epochs, use_cuda) # , external_res_attns

        train_fold_metrics_list.append(metrics_res_train)
        val_fold_metrics_list.append(metrics_res_val)
        independent_fold_metrics_list.append(metrics_res_independent)
        external_fold_metrics_list.append(metrics_res_external)

        ys_train_fold_dict[fold], ys_val_fold_dict[fold], ys_independent_fold_dict[fold], ys_external_fold_dict[fold] = ys_res_train, ys_res_val, ys_res_independent, ys_res_external    
        loss_train_fold_dict[fold], loss_val_fold_dict[fold], loss_independent_fold_dict[fold], loss_external_fold_dict[fold] = loss_res_train_list, loss_res_val_list, loss_res_independent_list, loss_res_external_list  

    print("Total training time: {:6.2f} sec".format(time_train))

=====Fold-0=====
-----Generate data loader-----
Fold-0 Label info: Train = Counter({1: 277903, 0: 277552}) | Val = Counter({0: 69607, 1: 69256})
-----Compile model-----
STSeqCls(
  (enc): StarTransEnc(
    (embedding): Embedding(21, 100, padding_idx=0)
    (emb_fc): Linear(in_features=100, out_features=300, bias=True)
    (encoder): StarTransformer(
      (norm): ModuleList(
        (0): LayerNorm((300,), eps=1e-06, elementwise_affine=True)
      )
      (emb_drop): Dropout(p=0.1, inplace=False)
      (ring_att): ModuleList(
        (0): _MSA1(
          (WQ): Conv2d(300, 256, kernel_size=(1, 1), stride=(1, 1))
          (WK): Conv2d(300, 256, kernel_size=(1, 1), stride=(1, 1))
          (WV): Conv2d(300, 256, kernel_size=(1, 1), stride=(1, 1))
          (WO): Conv2d(256, 300, kernel_size=(1, 1), stride=(1, 1))
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (star_att): ModuleList(
        (0): _MSA2(
          (WQ): Conv2d(300, 256, kernel_size=(1, 1), stride=(

100%|████████████████████████████████████████████████████████████████████████████████| 543/543 [00:53<00:00, 10.13it/s]


Fold-0****Train (Ep avg): Epoch-1/25 | Loss = 0.2670 | Time = 5.5459 sec
tn = 247570, fp = 29982, fn = 32362, tp = 245541
y_pred: 0 = 279932 | 1 = 275523
y_true: 0 = 277552 | 1 = 277903
auc=0.9563|sensitivity=0.8835|specificity=0.8920|acc=0.8878|mcc=0.7756
precision=0.8912|recall=0.8835|f1=0.8873|aupr=0.9582


100%|████████████████████████████████████████████████████████████████████████████████| 136/136 [00:06<00:00, 19.92it/s]


Fold-0 ****Test  Epoch-1/25: Loss = 0.180448
tn = 66866, fp = 2741, fn = 6586, tp = 62670
y_pred: 0 = 73452 | 1 = 65411
y_true: 0 = 69607 | 1 = 69256
auc=0.9804|sensitivity=0.9049|specificity=0.9606|acc=0.9328|mcc=0.8670
precision=0.9581|recall=0.9049|f1=0.9307|aupr=0.9819
****Saving model: Best epoch = 1 | 5metrics_Best_avg = 0.9277
*****Path saver:  G:TransPHLA-AOMP/model/STformer/netmhcpan/st_layer1_multihead8_fold0_netmhcpan.pkl


...

Fold-0****Train (Ep avg): Epoch-14/25 | Loss = 0.1034 | Time = 5.5838 sec
tn = 268349, fp = 9203, fn = 12456, tp = 265447
y_pred: 0 = 280805 | 1 = 274650
y_true: 0 = 277552 | 1 = 277903
auc=0.9930|sensitivity=0.9552|specificity=0.9668|acc=0.9610|mcc=0.9221
precision=0.9665|recall=0.9552|f1=0.9608|aupr=0.9935


100%|████████████████████████████████████████████████████████████████████████████████| 136/136 [00:06<00:00, 19.79it/s]


Fold-0 ****Test  Epoch-14/25: Loss = 0.148162
tn = 66010, fp = 3597, fn = 3340, tp = 65916
y_pred: 0 = 69350 | 1 = 69513
y_true: 0 = 69607 | 1 = 69256
auc=0.9876|sensitivity=0.9518|specificity=0.9483|acc=0.9500|mcc=0.9001
precision=0.9483|recall=0.9518|f1=0.9500|aupr=0.9883


  4%|███▍                                                                             | 23/543 [00:02<00:52,  9.92it/s]


KeyboardInterrupt: 

In [141]:
path_saver = '../model/st_layer1_multihead8_fold3_netmhcpan.pkl'
model = STSeqCls((21, 100), num_cls=2, hidden_size=300, num_layers=1, num_head=8, max_len=49,cls_hidden_size=600,dropout=0.1,head_dim=32).to(device)
model.load_state_dict(torch.load(path_saver))

<All keys matched successfully>

In [142]:
# model = model_eval
criterion = nn.CrossEntropyLoss()
fold = 3
ep_best = 7

model_eval = model.eval()
ys_res_independent, loss_res_independent_list, metrics_res_independent = eval_step(model_eval, independent_loader, fold, ep_best, epochs, use_cuda) # , independent_res_attns
ys_res_external, loss_res_external_list, metrics_res_external = eval_step(model_eval, external_loader, fold, ep_best, epochs, use_cuda) # , external_res_attns


100%|████████████████████████████████████████████████████████████████████████████████| 168/168 [00:08<00:00, 19.81it/s]


Fold-3 ****Test  Epoch-7/25: Loss = 0.367765
tn = 74424, fp = 11138, fn = 9313, tp = 76563
y_pred: 0 = 83737 | 1 = 87701
y_true: 0 = 85562 | 1 = 85876
auc=0.9444|sensitivity=0.8916|specificity=0.8698|acc=0.8807|mcc=0.7616
precision=0.8730|recall=0.8916|f1=0.8822|aupr=0.9448


100%|████████████████████████████████████████████████████████████████████████████████| 102/102 [00:05<00:00, 19.71it/s]


Fold-3 ****Test  Epoch-7/25: Loss = 0.336248
tn = 46839, fp = 5042, fn = 7528, tp = 44456
y_pred: 0 = 54367 | 1 = 49498
y_true: 0 = 51881 | 1 = 51984
auc=0.9410|sensitivity=0.8552|specificity=0.9028|acc=0.8790|mcc=0.7588
precision=0.8981|recall=0.8552|f1=0.8761|aupr=0.9472
